In [1]:
import matplotlib.pyplot as plt

# You may change the mhealth_activity module but your algorithm must support the original version
from mhealth_activity import Recording, Trace, Activity, WatchLocation, Path

import numpy as np

In [2]:
import os
import pandas as pd

files = os.listdir('./data/train')
list_of_dfs = []

for file in files:
    d = Recording(os.path.join('./data/train',file))
    list_of_dfs.append(d)

data = pd.DataFrame(list_of_dfs)

In [ ]:
data.columns = ["recording"]

In [ ]:
## Dataframe for acceleration data
acceleration = []
for row in data["recording"]:
    acceleration.extend([[row.data["ax"],row.data["ay"],row.data["az"]]])

acceleration = np.array(acceleration)

In [10]:
"""
accelerator_list_x = []
for sample in acceleration[:,0]:
    inside_list = []
    for x, t in list(zip(sample.timestamps, sample.values)):
        inside_list.append([x,t])
    accelerator_list_x.append(inside_list)
"""

In [ ]:
import scipy.stats as stats

FEATURES = ['MIN','MAX','MEAN','RMS','VAR','STD','POWER','PEAK','P2P','CREST FACTOR','SKEW','KURTOSIS',
            'MAX_f','SUM_f','MEAN_f','VAR_f','PEAK_f','SKEW_f','KURTOSIS_f']

In [ ]:
def features_extraction(df): 
    
    
    Min=[];Max=[];Mean=[];Rms=[];Var=[];Std=[];Power=[];Peak=[];Skew=[];Kurtosis=[];P2p=[];CrestFactor=[];
    FormFactor=[]; PulseIndicator=[];
    Max_f=[];Sum_f=[];Mean_f=[];Var_f=[];Peak_f=[];Skew_f=[];Kurtosis_f=[]
    
    X = df.values
    ## TIME DOMAIN ##

    Min.append(np.min(X))
    Max.append(np.max(X))
    Mean.append(np.mean(X))
    Rms.append(np.sqrt(np.mean(X**2)))
    Var.append(np.var(X))
    Std.append(np.std(X))
    Power.append(np.mean(X**2))
    Peak.append(np.max(np.abs(X)))
    P2p.append(np.ptp(X))
    CrestFactor.append(np.max(np.abs(X))/np.sqrt(np.mean(X**2)))
    Skew.append(stats.skew(X))
    Kurtosis.append(stats.kurtosis(X))
    FormFactor.append(np.sqrt(np.mean(X**2))/np.mean(X))
    PulseIndicator.append(np.max(np.abs(X))/np.mean(X))
    ## FREQ DOMAIN ##
    ft = fft(X)
    S = np.abs(ft**2)/len(df)
    Max_f.append(np.max(S))
    Sum_f.append(np.sum(S))
    Mean_f.append(np.mean(S))
    Var_f.append(np.var(S))
    
    Peak_f.append(np.max(np.abs(S)))
    Skew_f.append(stats.skew(X))
    Kurtosis_f.append(stats.kurtosis(X))
    #Create dataframe from features
    df_features = pd.DataFrame(index = [FEATURES], 
                               data = [Min,Max,Mean,Rms,Var,Std,Power,Peak,P2p,CrestFactor,Skew,Kurtosis,
                                       Max_f,Sum_f,Mean_f,Var_f,Peak_f,Skew_f,Kurtosis_f])
    return df_features

In [ ]:
inside_list = []
for x, t in list(zip(acceleration[0,1].timestamps, acceleration[0,1].values)):
    inside_list.append([x,t])
inside_list = pd.DataFrame(inside_list)
features_signal = features_extraction(inside_list)

In [ ]:
trial = pd.DataFrame.transpose(features_signal)

In [ ]:
from scipy.fft import fft, fftfreq

for sample in acceleration[1:,1]:
    new = []
    for x, t in list(zip(sample.timestamps, sample.values)):
        new.append([x,t])
    new = pd.DataFrame(new)
    features_inside = features_extraction(new)
    trial = pd.concat([trial,pd.DataFrame.transpose(features_inside)])

In [ ]:
trial.to_csv(f'extracted_signal_accelerator_y.csv')